# 🎙️ Hebrew Whisper Transcription Service

This notebook runs a Hebrew speech-to-text service using faster-whisper.

**Features:**
- GPU-accelerated transcription
- File upload transcription
- Live RTP/TCP audio transcription
- Hebrew-optimized ivrit-ai model

## 1️⃣ Setup & Installation

In [ ]:
# Install dependencies
!pip install -q faster-whisper python-dotenv numpy

# Check GPU
!nvidia-smi

In [ ]:
# Download the service files (or upload them manually)
# Option 1: Clone from your repo
# !git clone https://github.com/YOUR_REPO/whisper-standalone.git
# %cd whisper-standalone

# Option 2: Upload files manually using Colab's file upload

## 2️⃣ Initialize Transcriber

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from whisper_transcriber import WhisperTranscriber

# Initialize with GPU (use 'cpu' if no GPU)
transcriber = WhisperTranscriber(
    model_path="ivrit-ai/whisper-large-v3-turbo-ct2",
    device="cuda",  # or 'cpu'
    compute_type="float16",  # use 'int8' for CPU
)

# Load model (downloads on first run)
transcriber.load_model()
print("✅ Transcriber ready!")

## 3️⃣ Transcribe Files

In [ ]:
# Upload a file using Colab's file upload
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]
print(f"Uploaded: {filename}")

In [ ]:
# Transcribe the uploaded file
result = transcriber.transcribe_file(filename)

if result:
    print("\n" + "=" * 70)
    print(f"TRANSCRIPTION ({result.duration_seconds:.1f}s)")
    print("=" * 70)
    print(result.text)
    print("=" * 70)
    
    # Show segments
    if result.segments:
        print("\nSegments:")
        for i, seg in enumerate(result.segments, 1):
            print(f"  [{i}] {seg['start']:.1f}s - {seg['end']:.1f}s: {seg['text']}")

## 4️⃣ Live RTP Transcription (Optional)

Connect to your EC2 RTP relay for live radio transcription.

In [ ]:
# Configuration
RTP_HOST = "YOUR_EC2_IP"  # Replace with your EC2 relay IP
RTP_PORT = 5005

In [ ]:
import threading
import time
from rtp_client import RTPTCPClient

# Audio buffer
audio_buffer = []
buffer_lock = threading.Lock()
SAMPLE_RATE = 16000
BUFFER_DURATION = 4.0

def on_audio(audio_data, sample_rate):
    with buffer_lock:
        audio_buffer.append(audio_data)
        total = sum(len(c) // 2 for c in audio_buffer) / SAMPLE_RATE
        if total >= BUFFER_DURATION:
            process_buffer()

def process_buffer():
    global audio_buffer
    with buffer_lock:
        if not audio_buffer:
            return
        audio_bytes = b"".join(audio_buffer)
        audio_buffer = []
    
    result = transcriber.transcribe_audio(audio_bytes)
    if result and result.text:
        print(f"🎙️ {result.text}")

# Start RTP client
rtp_client = RTPTCPClient(
    host=RTP_HOST,
    port=RTP_PORT,
    audio_callback=on_audio,
)
rtp_client.start()
print(f"📡 Connected to {RTP_HOST}:{RTP_PORT}")
print("Listening for live audio...")

In [ ]:
# Run for a while, then stop
import time

try:
    print("Running... Press Stop (⏹️) to stop")
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    pass

rtp_client.stop()
print("Stopped.")

## 5️⃣ Batch Transcription

In [ ]:
# Transcribe multiple files
import glob

# Upload multiple files first, then:
audio_files = glob.glob("*.wav") + glob.glob("*.mp3")
print(f"Found {len(audio_files)} audio files")

for filepath in audio_files:
    print(f"\n📁 Processing: {filepath}")
    result = transcriber.transcribe_file(filepath)
    if result:
        print(f"📝 {result.text[:200]}..." if len(result.text) > 200 else f"📝 {result.text}")